In [1]:
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
import re, string, unicodedata
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import joblib

import time
nltk.download('stopwords')
sw = stopwords.words("english")

# 도서 text SVM

- 도서 설명 text 불러오기
- 각 도서 text에서 감정 추출 진행

**도서 text 텍스트 전처리**

- 텍스트 전처리 함수에 적용

In [2]:
def hapus_url(text):
    mention_pattern = r'@[\w]+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return re.sub(r'http\S+','', cleaned_text)
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text
def final_clean(text):
    final_text= []
    for i in text.split():
        if i.strip().lower() not in sw and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def clean(text):
    text = hapus_url(text)
    text = remove_special_characters(text, remove_digits=True)
    text = stemmer(text)
    text = final_clean(text)
    return text

In [3]:
# 예시 도서 text
text = '''끔찍한 실수로 아내를 죽이고 벽 안에 묻어버리는 주인공, 그런데 한때 자신이 애지중지한 고양이를 함께 가둬버리는 바람에 사람들에게 발각된다는 유명한 스토리를 한 번쯤 들어보았을 것이다. 쇠락해 가는 가문의 마지막 후계자로 고택과 함께 스러져 내리는 남매, 그저 자신을 바라보는 텅 빈 눈동자를 참을 수 없어 이웃집 노인을 살해하는 한 남자의 이야기는 어떤가? 에드거 앨런 포의 작품 세계는 어둡고 음침하다. 그러나 놀랍도록 세밀하고 탁월하다. 「검은 고양이」 「어셔가의 붕괴」 「고자질하는 심장」 등 공포 문학의 독보적인 선구자 에드거 앨런 포는 인간의 병약한 신경과 무질서한 정신세계를 치밀한 구조와 정교한 짜임새로 그린다. 그래서 기묘하고 충격적이면서도 어쩐지 포의 이야기는 읽는 이의 이해와 몰입을 불러일으킨다. 한편 포의 천재성은 최초의 탐정 소설이자 셜록 홈즈를 탄생시킨 「모르그가의 살인 사건」과 「도둑맞은 편지」 등 미스터리·추리소설에서도 빛을 발한다. 시초격이라고 할 수 있는 장르 문학에서조차 200년이 지난 현대극에서도 상상할 수 없는 반전이 도사리고 있다. 그런가 하면 풍자 소설에서는 재치와 기지가 돋보여서 「일주일에 세 번의 일요일」 「정확한 과학 중 하나로 여겨지는 사기술」 「타르 박사와 페더 교수의 치료법」을 읽으면서는 포의 기발한 상상과 익살에 웃음을 터트리게 된다. 포의 천재성은 굳이 말로 설명하지 않아도 그의 작품에 여실히 드러난다. 인생의 희극과 비극을 치열하게 겪은 작가, 그만큼 인간의 양면성과 기이한 심리를 섬세하게 포착한 작가, 정밀한 구도와 섬세한 필체로 아낌없이 풀어낸 그의 세계를 이 열 편의 엄선한 작품선집으로 담았다. 독자들에게 아름답고도 비극적이며 소름끼치는 광기의 미학을 선사할 것이다.'''

In [6]:
translator = Translator()
time.sleep(1)
text_t = translator.translate(text)
text_e = text_t.text
text_e = clean(text_e)
text_df = pd.DataFrame([text_e], columns = ['text'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
text_df['text']

0    main charact kill hi wife buri wall terribl mi...
Name: text, dtype: object

**도서 text에 SVM 모델 적용**

- 입력한 도서의 text 감정 벡터 추출 결과 = `book_sentiment`
- SVM 모델이 트위터 데이터로 학습했기 때문에 학습 당시 사용했던 input 형태와 동일한 형태로 현재 가사 text input을 변형해야함
- tf-idf 모델을 트위터 데이터로 fit_transform, 가사 text 데이터에 transform

In [9]:
path = 'text 감정 추출 모델'
df = pd.read_csv(path + '/tweet_data_agumentation.csv', index_col = 0)

model = joblib.load(path + '/SVM.pkl')
model

SVC(probability=True)

In [10]:
# df로 tf-idf 학습
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit_transform(df["content"])

# 학습한 tf-idf 모델을 도서 text에 동일하게 적용
text2 = tfidf_vect.transform(text_df['text'])
model.predict_proba(text2)

array([[0.00793652, 0.32936839, 0.01008081, 0.46980171, 0.08457674,
        0.01568081, 0.02345192, 0.04166963, 0.000539  , 0.01640461,
        0.00048987]])

In [12]:
book_sentiment = pd.DataFrame(model.predict_proba(text2), index=['prob']).T
book_sentiment['감정'] = ['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger']

book_sentiment = book_sentiment.sort_values(by='prob',ascending=False)
book_sentiment

,prob,감정
3,0.469802,worry
1,0.329368,sadness
4,0.084577,love
7,0.041670,happiness
6,0.023452,hate
9,0.016405,relief
5,0.015681,fun
2,0.010081,enthusiasm
0,0.007937,empty
8,0.000539,boredom
